# Recommender Systems 2021/22

### Practice - FunkSVD and SVD++ implemented with Python and Cython

FunkSVD is one of the simplest and most known matrix factorization models for rating prediction. It was proposed by Simon Funk in a now famous post on his website (with a cow grazing around Auckland): https://sifter.org/~simon/journal/20061211.html

SVD++ is an extension of FunkSVD that learns also the biases: global, user and item.

In [1]:
import time
import numpy as np

In [2]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

data_reader = Movielens10MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

Movielens10M: Preloaded data not found, reading from original files...
Movielens10M: Loading original data
Movielens10M: Unable to find data zip file. Downloading...
Downloading: https://files.grouplens.org/datasets/movielens/ml-10m.zip
In folder: /Users/jodyrobertobattistini/PycharmProjects/rec-sys-challenge/Data_manager/../Data_manager_split_datasets/Movielens10M/ml-10m.zip
DataReader: Downloaded 38.13%, 23.84 MB, 5466 KB/s, 4 seconds passed

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



DataReader: Downloaded 100.00%, 62.53 MB, 6670 KB/s, 10 seconds passed
Movielens10M: Loading Item Features Genres
Movielens10M: Loading Item Features Tags


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jodyrobertobattistini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Movielens10M: Loading Interactions
Movielens10M: Cleaning Temporary Files
Movielens10M: Loading Complete
Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
Movielens10M: Creating folder '/Users/jodyrobertobattistini/PycharmProjects/rec-sys-challenge/Data_manager/../Data_manager_split_datasets/Movielens10M/original/'
Movielens10M: Saving complete!
DataReader: current dataset is: Movielens10M
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_year, Value range: 1.92E+03 / 2.01E+03, Num features: 1, feature occurrences: 10681, density 1.00E+00
	ICM

In [3]:
URM_train

<69878x10681 sparse matrix of type '<class 'numpy.float64'>'
	with 8000043 stored elements in Compressed Sparse Row format>

### What do we need for FunkSVD and SVD++?

* Loss function
* User factor and Item factor matrices
* Computing prediction
* Update rule
* Training loop and some patience


In [4]:
n_users, n_items = URM_train.shape

#### The two methods are based on two latent factor matrices $ W \in R^{U \times E} , V \in R^{E \times I}$ with E the embedding size, and biases $ \mu \in R , b_u \in R^{U}, b_i \in R^{I}$

#### How to compute the predictions
FunkSVD: $ \hat{r}_{ui} = \sum_{j=0}^{E} W_{uj}H_{ji}$

SVD++: $ \hat{r}_{ui} = \mu + b_u + b_i + \sum_{j=0}^{E} W_{uj}H_{ji}$


#### The loss function we are interested in minimizing are

$L_{FunkSVD} = |R - WH|_F + \alpha|W|_2 + \beta|H|_2$

$L_{SVD++} = |R - (\mu + b_u + b_i + WH)|_F + \alpha|W|_2 + \beta|H|_2 + \gamma (\mu + b_u + b_i) $

Notice that in this case the loss is the Frobenius norm, not the 2 norm, hence we want to minimize the prediction error only for the existing ratings not the missing ones. In practice this means one samples only among the observed interactions.

While this approach works well for rating prediction, it does not for ranking. A model must be trained also on negative data if it expected to learn how to distinguish between positive and negative data. A good strategy is to randomly sample unobserved interactions during training assigning them a rating of 0.

#### Gradients

$\frac{\partial}{\partial W} L = -2(R - WH)H + 2\alpha W $

$\frac{\partial}{\partial H} L = -2(R - WH)W + 2\alpha H $

$\frac{\partial}{\partial \mu} L = -2(R - WH) + 2\gamma \mu $

$\frac{\partial}{\partial b_u} L = -2(R - WH) + 2\gamma b_u $

$\frac{\partial}{\partial b_i} L = -2(R - WH) + 2\gamma b_i $

#### The update is going to be (we can remove the coefficients)
$ W = W - \frac{\partial}{\partial W}$, or 

$ W = W + l((R - WH)H - \alpha W)$, with $l$ the learning rate

... and similarly for the other parameters: $H$, $\mu$, $b_u$ and $b_i$.

## Step 1: We create the dense latent factor matrices
### In a MF model you have two matrices, one with a row per user and the other with a column per item. The other dimension, columns for the first one and rows for the second one is called latent factors

In [5]:
num_factors = 10

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

In [6]:
user_factors

array([[0.93036035, 0.48943224, 0.53050326, ..., 0.45444864, 0.64604848,
        0.64109699],
       [0.94607056, 0.17130545, 0.61676869, ..., 0.75983268, 0.16593878,
        0.3230319 ],
       [0.07966808, 0.33538479, 0.57221528, ..., 0.0501624 , 0.53412466,
        0.98922084],
       ...,
       [0.69162328, 0.11788105, 0.48736927, ..., 0.89521311, 0.33903374,
        0.98055191],
       [0.23003903, 0.90119835, 0.94220865, ..., 0.67852993, 0.49616145,
        0.65477644],
       [0.38752484, 0.76558508, 0.81327766, ..., 0.55046864, 0.83754316,
        0.45137673]])

In [7]:
item_factors

array([[0.35537   , 0.64104503, 0.96289777, ..., 0.98117713, 0.41805847,
        0.23664264],
       [0.01140409, 0.67748615, 0.81098371, ..., 0.19387806, 0.22144021,
        0.73798123],
       [0.50135492, 0.60932249, 0.5399192 , ..., 0.06826006, 0.27701262,
        0.93081436],
       ...,
       [0.06123217, 0.29108337, 0.8131086 , ..., 0.41894751, 0.63334378,
        0.25505772],
       [0.06020495, 0.14662926, 0.35827009, ..., 0.20423205, 0.09524987,
        0.02346607],
       [0.42066259, 0.00311266, 0.20386564, ..., 0.83131245, 0.17120078,
        0.35281308]])

## Step 2: We sample an interaction and compute the prediction of the current FunkSVD model

In [8]:
URM_train_coo = URM_train.tocoo()

sample_index = np.random.randint(URM_train_coo.nnz)
sample_index

2863734

In [9]:
user_id = URM_train_coo.row[sample_index]
item_id = URM_train_coo.col[sample_index]
rating = URM_train_coo.data[sample_index]

(user_id, item_id, rating)

(25050, 1287, 3.0)

In [10]:
predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])
predicted_rating

4.098379193671592

#### The first predicted rating is a random prediction, essentially

### Step 3: We compute the prediction error and update the latent factor matrices

In [11]:
prediction_error = rating - predicted_rating
prediction_error

-1.098379193671592

### The error is positive, so we need to increase the prediction our model computes. Meaning, we have to increase the values latent factor matrices

### Which latent factors we modify? All the factors of the item and user we used

In [12]:
# Copy original value to avoid messing up the updates
H_i = item_factors[item_id,:]
W_u = user_factors[user_id,:]

In [13]:
H_i

array([0.5587849 , 0.05969929, 0.64214014, 0.6822657 , 0.32228913,
       0.50669907, 0.96267464, 0.91001872, 0.72677112, 0.9647236 ])

In [14]:
W_u

array([0.12091094, 0.24717702, 0.96734792, 0.9617698 , 0.75169658,
       0.31167153, 0.39781027, 0.541235  , 0.85755123, 0.87048519])

#### Apply the update rule

In [15]:
learning_rate = 1e-4
regularization = 1e-5

In [16]:
user_factors_update = prediction_error * H_i - regularization * W_u
user_factors_update

array([-0.61375892, -0.06557493, -0.70532304, -0.74939606, -0.35400319,
       -0.55655083, -1.05738577, -0.99955104, -0.79827886, -1.05964104])

In [17]:
item_factors_update = prediction_error * W_u - regularization * H_i
item_factors_update

array([-0.13281165, -0.27149469, -1.06252125, -1.05639476, -0.8256511 ,
       -0.34233859, -0.43695615, -0.59449036, -0.94192369, -0.95613247])

In [18]:
user_factors[user_id,:] += learning_rate * user_factors_update 
item_factors[item_id,:] += learning_rate * item_factors_update

### Let's check what the new prediction for the same user-item interaction would be

In [19]:
predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])
predicted_rating

4.097356533335718

### The value is higher than before, we are moving in the right direction

### And now? Sample another interaction and repeat... a lot of times

### WARNING: Initialization must be done with random non-zero values ... otherwise

In [20]:
user_factors = np.zeros((n_users, num_factors))
item_factors = np.zeros((n_items, num_factors))

predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])

print("Prediction is {:.2f}".format(predicted_rating))

prediction_error = rating - predicted_rating

print("Prediction error is {:.2f}".format(prediction_error))


Prediction is 0.00
Prediction error is 3.00


In [21]:
H_i = item_factors[item_id,:]
W_u = user_factors[user_id,:]

user_factors[user_id,:] += learning_rate * (prediction_error * H_i - regularization * W_u)
item_factors[item_id,:] += learning_rate * (prediction_error * W_u - regularization * H_i)

In [22]:
predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])

print("Prediction after the update is {:.2f}".format(predicted_rating))
print("Prediction error is {:.2f}".format(rating - predicted_rating))

Prediction after the update is 0.00
Prediction error is 3.00


### Since the matrices are multiplied, if we initialize one of them as zero, the updates will always be zero and the model will not be able to learn.

### Let's put all together in a training loop.

In [23]:
URM_train_coo = URM_train.tocoo()

num_factors = 10
learning_rate = 1e-4
regularization = 1e-5

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

loss = 0.0
start_time = time.time()

for sample_num in range(1000000):
    
    # Randomly pick sample
    sample_index = np.random.randint(URM_train_coo.nnz)

    user_id = URM_train_coo.row[sample_index]
    item_id = URM_train_coo.col[sample_index]
    rating = URM_train_coo.data[sample_index]

    # Compute prediction
    predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])
        
    # Compute prediction error, or gradient
    prediction_error = rating - predicted_rating
    loss += prediction_error**2
    
    # Copy original value to avoid messing up the updates
    H_i = item_factors[item_id,:]
    W_u = user_factors[user_id,:]  
    
    user_factors_update = prediction_error * H_i - regularization * W_u
    item_factors_update = prediction_error * W_u - regularization * H_i
    
    user_factors[user_id,:] += learning_rate * user_factors_update 
    item_factors[item_id,:] += learning_rate * item_factors_update    
    
    # Print some stats
    if (sample_num +1)% 100000 == 0:
        elapsed_time = time.time() - start_time
        samples_per_second = sample_num/elapsed_time
        print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

Iteration 100000 in 1.93 seconds, loss is 2.60. Samples per second 51932.59
Iteration 200000 in 3.84 seconds, loss is 2.56. Samples per second 52096.79
Iteration 300000 in 5.78 seconds, loss is 2.53. Samples per second 51932.04
Iteration 400000 in 7.71 seconds, loss is 2.51. Samples per second 51859.77
Iteration 500000 in 9.64 seconds, loss is 2.48. Samples per second 51877.02
Iteration 600000 in 11.56 seconds, loss is 2.45. Samples per second 51898.52
Iteration 700000 in 13.49 seconds, loss is 2.43. Samples per second 51897.02
Iteration 800000 in 15.47 seconds, loss is 2.41. Samples per second 51723.54
Iteration 900000 in 17.39 seconds, loss is 2.39. Samples per second 51752.04
Iteration 1000000 in 19.34 seconds, loss is 2.37. Samples per second 51718.31


### What do we see? The loss generally goes down but may oscillate a bit.
### How long do we train such a model?

* An epoch: a complete loop over all the train data
* Usually you train for multiple epochs. Depending on the algorithm and data 10s or 100s of epochs.

In [24]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 1546.84 seconds, or 25.78 minutes


### This model is relatively quick

### Let's see what we can do with Cython
### First step, just compile it. We do not have the data at compile time, so we put the loop in a function

In [25]:
%load_ext Cython

In [26]:
%%cython
import numpy as np
import time

def do_some_training(URM_train):

    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape

    num_factors = 10
    learning_rate = 1e-4
    regularization = 1e-5

    user_factors = np.random.random((n_users, num_factors))
    item_factors = np.random.random((n_items, num_factors))

    loss = 0.0
    start_time = time.time()

    for sample_num in range(1000000):

        # Randomly pick sample
        sample_index = np.random.randint(URM_train_coo.nnz)

        user_id = URM_train_coo.row[sample_index]
        item_id = URM_train_coo.col[sample_index]
        rating = URM_train_coo.data[sample_index]

        # Compute prediction
        predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])

        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error**2

        # Copy original value to avoid messing up the updates
        H_i = item_factors[item_id,:]
        W_u = user_factors[user_id,:]  

        user_factors_update = prediction_error * H_i - regularization * W_u
        item_factors_update = prediction_error * W_u - regularization * H_i

        user_factors[user_id,:] += learning_rate * user_factors_update 
        item_factors[item_id,:] += learning_rate * item_factors_update    

        # Print some stats
        if (sample_num +1)% 100000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

    return loss, samples_per_second

Content of stderr:
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_2d8c9f90376a38eb682a7450b616ee756322867f.c:5200:21: warning: fallthrough annotation in unreachable code [-Wunreachable-code-fallthrough]
                    CYTHON_FALLTHROUGH;
                    ^
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_2d8c9f90376a38eb682a7450b616ee756322867f.c:530:34: note: expanded from macro 'CYTHON_FALLTHROUGH'
      #define CYTHON_FALLTHROUGH __attribute__((fallthrough))
                                 ^
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_2d8c9f90376a38eb682a7450b616ee756322867f.c:5211:21: warning: fallthrough annotation in unreachable code [-Wunreachable-code-fallthrough]
                    CYTHON_FALLTHROUGH;
                    ^
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_2d8c9f90376a38eb682a7450b616ee756322867f.c:530:34: note: expanded from macro 'CYTHON_FALLTHROUGH'
      #define CYTHON_FALLTHROUGH __attribute__((fallth

In [27]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 100000 in 2.24 seconds, loss is 2.62. Samples per second 44639.96
Iteration 200000 in 3.97 seconds, loss is 2.61. Samples per second 50333.10
Iteration 300000 in 5.76 seconds, loss is 2.58. Samples per second 52099.69
Iteration 400000 in 7.55 seconds, loss is 2.55. Samples per second 52995.55
Iteration 500000 in 9.32 seconds, loss is 2.52. Samples per second 53640.46
Iteration 600000 in 11.13 seconds, loss is 2.49. Samples per second 53893.23
Iteration 700000 in 12.90 seconds, loss is 2.47. Samples per second 54280.93
Iteration 800000 in 14.63 seconds, loss is 2.45. Samples per second 54686.36
Iteration 900000 in 16.39 seconds, loss is 2.43. Samples per second 54912.45
Iteration 1000000 in 18.17 seconds, loss is 2.41. Samples per second 55036.91


In [28]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 1453.57 seconds, or 24.23 minutes


### The compiler is just porting in C all operations that the python interpreter would have to perform, dynamic tiping included. Have a look at the html reports in the Cython_examples folder

### Now try to add some types: If you use a variable only as a C object, use primitive tipes

* cdef int namevar
* cdef double namevar
* cdef float namevar
* cdef double[:] singledimensionarray
* cdef double[:,:] bidimensionalmatrix

### Some operations are still done with sparse matrices, those cannot be correctly optimized because the compiler does not know how what is the type of the data.

### To address this, we create typed arrays in which we put the URM_train data
For example, this operation: user_id = URM_train_coo.row[sample_index]

Becomes:
cdef int user_id
cdef int[:] URM_train_coo_row = URM_train_coo.row
user_id = URM_train_coo_row[sample_index]

### We can also skip the creation of the items_in_user_profile array and replace the np.random call with the faster native C function rand()


### We now use types for all main variables


In [29]:
%%cython
import numpy as np
import time

from libc.stdlib cimport rand, srand, RAND_MAX

def do_some_training(URM_train):

    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape
    cdef int n_interactions = URM_train.nnz
    
    cdef int sample_num, sample_index, user_id, item_id, factor_index
    cdef double rating, predicted_rating, prediction_error

    cdef int num_factors = 10
    cdef double learning_rate = 1e-4
    cdef double regularization = 1e-5
    
    cdef int[:] URM_train_coo_row = URM_train_coo.row
    cdef int[:] URM_train_coo_col = URM_train_coo.col
    cdef double[:] URM_train_coo_data = URM_train_coo.data
    
    cdef double[:,:] user_factors = np.random.random((n_users, num_factors))
    cdef double[:,:] item_factors = np.random.random((n_items, num_factors))
    cdef double H_i, W_u
    cdef double item_factors_update, user_factors_update
                
    cdef double loss = 0.0
    cdef long start_time = time.time()

    for sample_num in range(URM_train.nnz):

        # Randomly pick sample
        sample_index = rand() % n_interactions

        user_id = URM_train_coo_row[sample_index]
        item_id = URM_train_coo_col[sample_index]
        rating = URM_train_coo_data[sample_index]

        # Compute prediction
        predicted_rating = 0.0
        
        for factor_index in range(num_factors):
            predicted_rating += user_factors[user_id, factor_index] * item_factors[item_id, factor_index]
 
        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error**2

        # Copy original value to avoid messing up the updates
        for factor_index in range(num_factors):
            
            H_i = item_factors[item_id,factor_index]
            W_u = user_factors[user_id,factor_index]  

            user_factors_update = prediction_error * H_i - regularization * W_u
            item_factors_update = prediction_error * W_u - regularization * H_i

            user_factors[user_id,factor_index] += learning_rate * user_factors_update 
            item_factors[item_id,factor_index] += learning_rate * item_factors_update    

        # Print some stats
        if (sample_num +1)% 500000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

    return loss, samples_per_second

Content of stderr:
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_4066a0420c9d9b55cfe53e13f942763025db5a1c.c:22933:26: warning: code will never be executed [-Wunreachable-code]
                module = PyImport_ImportModuleLevelObject(
                         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 warning generated.
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_4066a0420c9d9b55cfe53e13f942763025db5a1c.c:22933:26: warning: code will never be executed [-Wunreachable-code]
                module = PyImport_ImportModuleLevelObject(
                         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 warning generated.

In [30]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 500000 in 0.50 seconds, loss is 2.44. Samples per second 993948.48
Iteration 1000000 in 0.64 seconds, loss is 2.34. Samples per second 1564911.70
Iteration 1500000 in 0.93 seconds, loss is 2.25. Samples per second 1619813.66
Iteration 2000000 in 1.30 seconds, loss is 2.19. Samples per second 1533497.19
Iteration 2500000 in 1.54 seconds, loss is 2.13. Samples per second 1619487.28
Iteration 3000000 in 1.69 seconds, loss is 2.08. Samples per second 1776323.58
Iteration 3500000 in 1.83 seconds, loss is 2.04. Samples per second 1917155.43
Iteration 4000000 in 1.96 seconds, loss is 2.00. Samples per second 2042378.71
Iteration 4500000 in 2.10 seconds, loss is 1.97. Samples per second 2147580.30
Iteration 5000000 in 2.24 seconds, loss is 1.94. Samples per second 2235765.70
Iteration 5500000 in 2.37 seconds, loss is 1.91. Samples per second 2318344.67
Iteration 6000000 in 2.51 seconds, loss is 1.88. Samples per second 2392238.22
Iteration 6500000 in 2.65 seconds, loss is 1.86. Sampl

In [31]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 30.51 seconds, or 0.51 minutes


### Works nicely, let's put an additional for loop to do multiple epochs

In [32]:
%%cython
import numpy as np
import time

from libc.stdlib cimport rand, srand, RAND_MAX

def train_multiple_epochs(URM_train, learning_rate_input, regularization_input, n_epochs):
    
    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape
    cdef int n_interactions = URM_train.nnz
    
    cdef int sample_num, sample_index, user_id, item_id, factor_index
    cdef double rating, predicted_rating, prediction_error

    cdef int num_factors = 10
    cdef double learning_rate = learning_rate_input
    cdef double regularization = regularization_input
    
    cdef int[:] URM_train_coo_row = URM_train_coo.row
    cdef int[:] URM_train_coo_col = URM_train_coo.col
    cdef double[:] URM_train_coo_data = URM_train_coo.data

    cdef double[:,:] user_factors = np.random.random((n_users, num_factors))
    cdef double[:,:] item_factors = np.random.random((n_items, num_factors))
    cdef double H_i, W_u
    cdef double item_factors_update, user_factors_update
                
    cdef double loss = 0.0
    cdef long start_time = time.time()
    
    for n_epoch in range(n_epochs):

        loss = 0.0
        start_time = time.time()

        for sample_num in range(URM_train.nnz):

            # Randomly pick sample
            sample_index = rand() % n_interactions

            user_id = URM_train_coo_row[sample_index]
            item_id = URM_train_coo_col[sample_index]
            rating = URM_train_coo_data[sample_index]

            # Compute prediction
            predicted_rating = 0.0

            for factor_index in range(num_factors):
                predicted_rating += user_factors[user_id, factor_index] * item_factors[item_id, factor_index]

            # Compute prediction error, or gradient
            prediction_error = rating - predicted_rating
            loss += prediction_error**2

            # Copy original value to avoid messing up the updates
            for factor_index in range(num_factors):

                H_i = item_factors[item_id,factor_index]
                W_u = user_factors[user_id,factor_index]  

                user_factors_update = prediction_error * H_i - regularization * W_u
                item_factors_update = prediction_error * W_u - regularization * H_i

                user_factors[user_id,factor_index] += learning_rate * user_factors_update 
                item_factors[item_id,factor_index] += learning_rate * item_factors_update    
            
        elapsed_time = time.time() - start_time
        samples_per_second = sample_num/elapsed_time
     
        print("Epoch {} complete in in {:.2f} seconds, loss is {:.3E}. Samples per second {:.2f}".format(n_epoch+1, time.time() - start_time, loss/sample_num, samples_per_second))

    return np.array(user_factors), np.array(item_factors), loss, samples_per_second    

Content of stderr:
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_9a1303b32c8f966755c4784110fb0e619e65e676.c:23231:26: warning: code will never be executed [-Wunreachable-code]
                module = PyImport_ImportModuleLevelObject(
                         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_9a1303b32c8f966755c4784110fb0e619e65e676.c:23746:21: warning: fallthrough annotation in unreachable code [-Wunreachable-code-fallthrough]
                    CYTHON_FALLTHROUGH;
                    ^
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_9a1303b32c8f966755c4784110fb0e619e65e676.c:533:34: note: expanded from macro 'CYTHON_FALLTHROUGH'
      #define CYTHON_FALLTHROUGH __attribute__((fallthrough))
                                 ^
/Users/jodyrobertobattistini/.ipython/cython/_cython_magic_9a1303b32c8f966755c4784110fb0e619e65e676.c:23757:21: warning: fallthrough annotation in unreachable code [-Wunreachable-co

In [33]:
n_items = URM_train.shape[1]
learning_rate = 1e-3
regularization = 1e-5
    
user_factors, item_factors, loss, samples_per_second =  train_multiple_epochs(URM_train, learning_rate, regularization, 10)

Epoch 1 complete in in 2.54 seconds, loss is 1.172E+00. Samples per second 3154364.41
Epoch 2 complete in in 2.67 seconds, loss is 8.915E-01. Samples per second 2994942.51
Epoch 3 complete in in 2.82 seconds, loss is 8.307E-01. Samples per second 2841360.89
Epoch 4 complete in in 3.37 seconds, loss is 8.009E-01. Samples per second 2376559.85
Epoch 5 complete in in 2.49 seconds, loss is 7.839E-01. Samples per second 3213890.54
Epoch 6 complete in in 2.63 seconds, loss is 7.731E-01. Samples per second 3042099.06
Epoch 7 complete in in 2.77 seconds, loss is 7.654E-01. Samples per second 2887246.95
Epoch 8 complete in in 2.91 seconds, loss is 7.599E-01. Samples per second 2753362.18
Epoch 9 complete in in 3.04 seconds, loss is 7.562E-01. Samples per second 2627521.08
Epoch 10 complete in in 2.18 seconds, loss is 7.528E-01. Samples per second 3661654.60


### From 20 minutes of training time to a few seconds...